In [ ]:
root_dir = "/content"
runtime_dir = !pwd
if runtime_dir[0] != root_dir:
    raise Exception("Runtime folder does not match root folder.")
!rm -rf *
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install roboflow
!pip install ultralytics==8.0.196

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 17.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.3 MB/s eta 0:00:00

In [ ]:
import torch
import logging

orig_torch_load = torch.load

def torch_wrapper(*args, **kwargs):
    kwargs['weights_only'] = False

    return orig_torch_load(*args, **kwargs)

torch.load = torch_wrapper

NODE_CLASS_MAPPINGS = {}
__all__ = ['NODE_CLASS_MAPPINGS']

In [ ]:
from roboflow import Roboflow
from ultralytics import YOLO
import numpy as np
import os
import cv2

In [ ]:
from ultralytics import settings
settings.update({"wandb": False})

In [ ]:
rf = Roboflow(api_key="{SECRET}")
project = rf.workspace("tfm-ks6ji").project("disc-detection-v2-zqrln")
version = project.version(1)
dataset = version.download("yolov8")

!mv Disc-detection-v2-$version.version "00_labeled_dataset"

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Disc-detection-v2-1 in yolov8:: 100%|██████████| 1491/1491 [00:00<00:00, 8252.15it/s]


In [ ]:
model = YOLO("yolov8n-seg.pt")
result = model.train(
    data = os.path.join(root_dir, "00_labeled_dataset", "data.yaml"),
    epochs = 100,
    imgsz = 300,
    batch = 32,
    name = "weakly_supervised_disk",
    device = "0",
)

100%|██████████| 6.73M/6.73M [00:00<00:00, 77.7MB/s]
New https://pypi.org/project/ultralytics/8.3.143 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/00_labeled_dataset/data.yaml, epochs=100, patience=50, batch=32, imgsz=300, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=weakly_supervised_disk, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, v

In [ ]:
def compute_iou(pred_mask, true_mask):
    intersection = (pred_mask & true_mask).float().sum((1, 2))
    union = (pred_mask | true_mask).float().sum((1, 2))
    iou = (intersection + 1e-6) / (union + 1e-6)
    return iou

def compute_dice(pred_mask, true_mask):
    intersection = (pred_mask & true_mask).float().sum((1, 2))
    dice = (2. * intersection + 1e-6) / (pred_mask.float().sum((1, 2)) + true_mask.float().sum((1, 2)) + 1e-6)
    return dice

def polygon_to_mask(coords, height, width):
    coords = np.array(coords, dtype=np.float32)
    coords = coords.reshape((-1, 1, 2)) * [width, height]
    coords = coords.astype(np.int32)

    mask = np.zeros((height, width), dtype=np.uint8)
    cv2.fillPoly(mask, [coords], 1)
    return torch.tensor(mask, dtype=torch.bool)

In [ ]:
n = 0
inference_time = 0
ious = []
dices = []
for root, dirs, files in os.walk("/content/00_labeled_dataset/test/images"):
  for file in files:
    path = os.path.join(root, file)
    result = model.predict(path)[0]
    inference_time += result.speed['inference']
    with open(os.path.join(root, file).replace("images", "labels").replace(".jpg", ".txt")) as f:
      true_label = f.read()
      true_label = true_label.split(" ")

    label = []
    for coords in result.masks.xyn[0]:
      label.append(coords[0])
      label.append(coords[1])
    del true_label[0]

    iou = compute_iou(polygon_to_mask(label, 320, 320).unsqueeze(0), polygon_to_mask(true_label,320,320).unsqueeze(0))
    dice = compute_dice(polygon_to_mask(label, 320, 320).unsqueeze(0), polygon_to_mask(true_label,320,320).unsqueeze(0))

    ious.append(iou.item())
    dices.append(dice.item())

print("IOU: {} | DICE: {}.".format(np.mean(ious), np.mean(dices)))
print("Inference time: {} ms.".format(inference_time/len(ious)))


image 1/1 /content/00_labeled_dataset/test/images/casting_data_train_cast_ok_cast_ok_0_7037_jpeg_jpg.rf.b44f263d79022d39ed2542a3c1bfd184.jpg: 320x320 1 discs, 8.7ms
Speed: 1.2ms preprocess, 8.7ms inference, 5.0ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 /content/00_labeled_dataset/test/images/casting_data_train_cast_ok_cast_ok_0_1550_jpeg_jpg.rf.57b4dd8337d162d1a2d0c6cc65701bd4.jpg: 320x320 1 discs, 9.6ms
Speed: 1.1ms preprocess, 9.6ms inference, 4.7ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 /content/00_labeled_dataset/test/images/casting_data_train_cast_ok_cast_ok_0_7127_jpeg_jpg.rf.fb11ec5ea58eb01572f6b376adc4163e.jpg: 320x320 1 discs, 12.0ms
Speed: 1.1ms preprocess, 12.0ms inference, 5.2ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 /content/00_labeled_dataset/test/images/casting_data_train_cast_ok_cast_ok_0_5522_jpeg_jpg.rf.b847adfaa94604df3efe600f7ae7872d.jpg: 320x320 1 discs, 9.3ms
Speed: 1.1ms preprocess, 9.3ms inference, 4.2m

IOU: 0.9849537312984467 | DICE: 0.9924186706542969.
Inference time: 9.366917610168457 ms.


In [ ]:
# Deploy to Roboflow.
project.version(dataset.version).deploy(model_type="yolov8", model_path=result.save_dir)

View the status of your deployment at: https://app.roboflow.com/tfm-ks6ji/disc-detection-v2/1
